# Parallel computing with Julia
Julia has been designed to allow the users to paralellize their code using of simple syntax.

**Parallel computing:** Divide a problem into smaller, similar problems that can be solved independently by different computation units. 

## Shared memory parallelization

### The "@threads" macro
The parallelization of a for loop is one of the most used schemes in parallel computing.
- Divide the data into chunks of similar size (static scheduling).
- Assign each chunk of data to (ideally) a different core.
- Each core performs the evaluations inside the for loop on its corresponding chunk of data.

In [14]:
#First example: parallel for
Threads.@threads for i = 1:Threads.nthreads()
    println("Hello from thread ",Threads.threadid())
end

Hello from thread 1


The number of threads can be defined when the program is launched:   
  
julia -t <no. of threads> < code file>  
  
or with the environment variable JULIA_NUM_THREADS

In [55]:
!submit_julia 'julia -t 2 threads/hello_world_thr.jl'

srun: job 141203 queued and waiting for resources
srun: job 141203 has been allocated resources
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Hello from thread 1
Hello from thread 2


------

In [ ]:
#Perform a calculation "foo" on each element of an array
function calc!(v)
    for i = 1:length(v)
        v[i] = foo(v[i]) 
    end
end

Example:

In [33]:
!submit_julia1 'julia threads/example1.jl 100000'

srun: job 141121 queued and waiting for resources
srun: job 141121 has been allocated resources
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
  1.520132 seconds (37.18 k allocations: 2.435 MiB, 0.78% compilation time)


Parallelize the for loop with @threads macro. This divides the data automatically so that each thread gets a chunk and performs the calculation on its assigned elements.

In [ ]:
#Parallelized for loop
function calc!(v)
    Threads.@threads for i = 1:length(v)
        v[i] = foo(v[i]) 
    end
end

Parallelized example:

In [41]:
!submit_julia1 'julia -t 32 threads/example1_par.jl 100000'

#!/usr/bin/bash
export PATH=/opt/ohpc/pub/compiler/julia/1.6.0/bin:$PATH
export LD_LIBRARY_PATH=/opt/ohpc/pub/compiler/julia/1.6.0/lib64:$LD_LIBRARY_PATH
julia -t 32 threads/example1_par.jl 100000
srun: job 141139 queued and waiting for resources
srun: job 141139 has been allocated resources
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
  0.156067 seconds (106.53 k allocations: 6.724 MiB, 22.38% compilation time)


### Race condition
When several threads try to access the same value at the same time undefined behavior may occur.

In [ ]:
#Add the elements of v
function add(v)
    res = 0
    for i = 1:length(v)
        res += v[i]
    end
    return res
end

Example:

In [15]:
!submit_julia 'julia threads/example2.jl 1000000'

srun: job 140944 queued and waiting for resources
srun: job 140944 has been allocated resources
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warni

Parallelize the for loop with @threads.

In [ ]:
#Parallelize
function add(v)
    res = 0
    Threads.@threads for i = 1:length(v)
        res += v[i]
    end
    return res
end

Parallelized example:

In [16]:
!submit_julia 'julia -t 4 threads/example2_par_bad.jl 1000000'

srun: job 140945 queued and waiting for resources
srun: job 140945 has been allocated resources
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warni

Test Failed at /datacnmat01/macc/usuarios/carlosedu.alvarez/julia/workshop/threads/example2_par_bad.jl:20
  Expression: r == N
   Evaluated: 250000.0 == 1000000
Test Failed at /datacnmat01/macc/usuarios/carlosedu.alvarez/julia/workshop/threads/example2_par_bad.jl:20
  Expression: r == N
   Evaluated: 250000.0 == 1000000
Test Failed at /datacnmat01/macc/usuarios/carlosedu.alvarez/julia/workshop/threads/example2_par_bad.jl:20
  Expression: r == N
   Evaluated: 250707.0 == 1000000
Test Failed at /datacnmat01/macc/usuarios/carlosedu.alvarez/julia/workshop/threads/example2_par_bad.jl:20
  Expression: r == N
   Evaluated: 507913.0 == 1000000
ERROR: LoadError: There was an error during testing
in expression starting at /datacnmat01/macc/usuarios/carlosedu.alvarez/julia/workshop/threads/example2_par_bad.jl:20
Test Failed at /datacnmat01/macc/usuarios/carlosedu.alvarez/julia/workshop/threads/example2_par_bad.jl:20
  Expression: r == N
   Evaluated: 250000.0 == 1000000
srun: error: thanatos-1-7:

What is wrong with the result?  

Give each thread a separate space in memory to accumulate its results.

In [15]:
#Each thread writes on a different element of the array
function add(v)
    partial_res = zeros(Threads.nthreads())
    Threads.@threads for i = 1:length(v)
        partial_res[Threads.threadid()] += v[i]
    end
    return sum(partial_res)
end

add (generic function with 1 method)

Correctly parallelized example: 

In [17]:
!submit_julia 'julia -t 4 threads/example2_par_ok.jl 1000000'

srun: job 140946 queued and waiting for resources
srun: job 140946 has been allocated resources
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warni

The result is correct now.

**Exercise 6 (20 min):**
- Parallelize the vector sum using @threads.
- Parallelize the vector scalar product using @threads.
- Parallelize the matrix product using @threads. (Try paralelizing different loops)

### The "@spawn" macro
- Create a new thread to execute a particular task
- More efficient than @threads when dealing with non homogeneous tasks

In [17]:
Threads.@sync for i = 1:Threads.nthreads()
    Threads.@spawn println("Hello from thread ",Threads.threadid())
end

Hello from thread 1


Example:

In [18]:
!submit_julia 'julia -t 4 threads/hello_world_spn.jl'

srun: job 140947 queued and waiting for resources
srun: job 140947 has been allocated resources
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warni

**Exercise 7** (20 min): Compute the total potential energy of a system of particles.  
As most of the time the potential is pair-wise $u_{ij}=u(x_i,x_j)$, for a system with $N$ particles we make a sum of the type:
$$U=\sum_{i=1}^{N-1}\sum_{j=i+1}^Nu_{ij}$$

In [ ]:
#Compute the interaction energy between all the particles whose positions
# are stored in v
function potential(v)
    U = 0
    for i = 1:length(v)-1
        for j = i+1:length(v)
            U += lennard_jones(v[i],v[j])
        end
    end
    return U
end

Serial code:

In [57]:
!submit_julia 'julia threads/potential_add_ser.jl 10000'

srun: job 141246 queued and waiting for resources
srun: job 141246 has been allocated resources
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
  4.854178 seconds (31.04 k allocations: 1.934 MiB, 0.51% compilation time)
-1.0170953392953207e-12


Parallelize the code using @threads as well as @spawn. Compare the speedup.

----

**Exercise 8** (20 min): Compute the Mandelbrot set.  
Consider the function $f_c(z)=z^2+c$. The Mandelbrot set consists of the points $c$ on the complex plane for which the sequence  
$$f_c(z),f_c(f_c(z)),\cdots$$
does not diverge when iterated from $z=0$.

Go over the complex plane classifying points according to whether they are in the Mandelbrot set or not.

In [ ]:
#Check whether or not the point c is in the mandelbrot set
function is_in(c)
    z = 0 + 0im
    for i = 1:MAX_ITER
        z = z*z + c
        if abs2(z) > 4
            return false
        end
    end
    return true
end

Serial code:

In [58]:
!submit_julia 'julia threads/mandelbrot_ser.jl 2000 3000'

srun: job 141249 queued and waiting for resources
srun: job 141249 has been allocated resources
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
  6.308816 seconds (136.62 k allocations: 8.578 MiB, 0.74% compilation time)
Area: 1.510617


Parallelize the code using @threads as well as @spawn. Compare the speedup.